In [1]:
from pprint import pprint

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from prepare import basic_clean, lemmatize

In [2]:
"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = [
    "gocodeup/codeup-setup-script",
    "gocodeup/movies-application",
    "torvalds/linux",
]

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        return repo_info.get("language", None)
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = None
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data() -> List[Dict[str, str]]:
    """
    Loop through all of the repos and process them. Returns the processed data.
    """
    return [process_repo(repo) for repo in REPOS]


if __name__ == "__main__":
    documents = scrape_github_data()
    json.dump(documents, open("data.json", "w"), indent=1)

In [5]:
documents

[{'repo': 'gocodeup/codeup-setup-script',
  'language': 'Shell',
  'readme_contents': '# Codeup Setup Script\n\nSetup script for Codeup students\' laptops to install the all the tools we will\nneed for the java course. We will install the following:\n\n- [xcode](https://developer.apple.com/xcode/features/): command line tools for\n  macs\n- [brew](http://brew.sh/): package manager for macs\n- [java](https://en.wikipedia.org/wiki/Java_(programming_language))\n- [tomcat](http://tomcat.apache.org/): the java webserver\n- [maven](https://maven.apache.org/): a java dependency and build management tool\n- [mysql](https://www.mysql.com/): the database we\'ll use for the class\n- [node js](https://nodejs.org/en/): a JavaScript runtime outside of the browser\n- [npm](https://www.npmjs.com/): a package manager for JavaScript\n\nIn addition, we will:\n\n- setup ssh keys for the student\'s laptop and guide them through the process of\n  linking their ssh key to their Github account.\n- Setup a glo

In [38]:
len(documents)

documents[0]

{'repo': 'gocodeup/codeup-setup-script',
 'language': 'Shell',
 'readme_contents': '# Codeup Setup Script\n\nSetup script for Codeup students\' laptops to install the all the tools we will\nneed for the java course. We will install the following:\n\n- [xcode](https://developer.apple.com/xcode/features/): command line tools for\n  macs\n- [brew](http://brew.sh/): package manager for macs\n- [java](https://en.wikipedia.org/wiki/Java_(programming_language))\n- [tomcat](http://tomcat.apache.org/): the java webserver\n- [maven](https://maven.apache.org/): a java dependency and build management tool\n- [mysql](https://www.mysql.com/): the database we\'ll use for the class\n- [node js](https://nodejs.org/en/): a JavaScript runtime outside of the browser\n- [npm](https://www.npmjs.com/): a package manager for JavaScript\n\nIn addition, we will:\n\n- setup ssh keys for the student\'s laptop and guide them through the process of\n  linking their ssh key to their Github account.\n- Setup a global

In [43]:
stuff = {text: document['readme_contents'] for document in documents}

NameError: name 'text' is not defined

In [34]:
print('\nCleaning and lemmatizing...\n')

documents = {text: lemmatize(basic_clean(topic[2])) for topic in documents}

pprint(documents)


Cleaning and lemmatizing...



KeyError: 2

In [27]:
# A simple way to calculate idf for demonstration. Note that this
# function relies on the globally defined documents variable.
def idf(word):
    n_occurences = sum([1 for doc in documents.values() if word in doc])
    return len(documents) / n_occurences

# Get a list of the unique words
unique_words = pd.Series(' '.join(words.split()).unique())

# put the unique words into a data frame
(pd.DataFrame(dict(word=unique_words))
 # calculate the idf for each word
 .assign(idf=lambda df: df.word.apply(idf))
 # sort the data for presentation purposes
 .set_index('word')
 .sort_values(by='idf', ascending=False)
 .head(5))

AttributeError: 'str' object has no attribute 'unique'